In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
!pip install rwkv

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.2/406.2 kB 7.4 MB/s eta 0:00:00


In [ ]:
print(os.getcwd())

/content


In [ ]:
# !wget https://huggingface.co/BlinkDL/rwkv-6-world/resolve/main/RWKV-x060-World-3B-v2-20240228-ctx4096.pth
!wget https://huggingface.co/BlinkDL/temp-latest-training-models/resolve/main/RWKV-x060-chn_single_round_test-1B6-20240427-ctx1024.pth

--2024-04-28 10:36:23--  https://huggingface.co/BlinkDL/temp-latest-training-models/resolve/main/RWKV-x060-chn_single_round_test-1B6-20240427-ctx1024.pth
Resolving huggingface.co (huggingface.co)... 3.163.189.114, 3.163.189.37, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/20/96/209600910186b29a71a230a87a5a555fd33bc07c57dabd6f6f9fd03523c5326f/3d088400933a96187c2b19e8e04072a70b881ead563ecb8e6a4c866c88a0b811?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27RWKV-x060-chn_single_round_test-1B6-20240427-ctx1024.pth%3B+filename%3D%22RWKV-x060-chn_single_round_test-1B6-20240427-ctx1024.pth%22%3B&Expires=1714559783&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNDU1OTc4M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8yMC85Ni8yMDk2MDA5MTAxODZiMjlhNzFhMjMwYTg3YTVhNTU1Z

In [ ]:
!pip install ninja tqdm ftfy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 7.3 MB/s eta 0:00:00


In [ ]:
########################################################################################################
# The RWKV Language Model - https://github.com/BlinkDL/RWKV-LM
########################################################################################################

print("RWKV Chat Simple Demo")

import os, copy, types, gc, sys, re, json, csv
import numpy as np
from copy import deepcopy
# from prompt_toolkit import prompt
import torch

torch.backends.cudnn.benchmark = True
torch.backends.cudnn.allow_tf32 = True
torch.backends.cuda.matmul.allow_tf32 = True
os.environ["RWKV_JIT_ON"] = "1"
os.environ["RWKV_CUDA_ON"] = "1"  # '1' to compile CUDA kernel (10x faster), requires c++ compiler & cuda libraries

from rwkv.model import RWKV
from rwkv.utils import PIPELINE

########################################################################################################

args = types.SimpleNamespace()

args.strategy = "cuda fp16"  # use CUDA, fp16

args.MODEL_NAME = "RWKV-x060-chn_single_round_test-1B6-20240427-ctx1024"

GEN_TEMP = 1.0
GEN_TOP_P = 0.3
GEN_alpha_presence = 0.0
GEN_alpha_frequency = 1.0
GEN_penalty_decay = 0.996

CHUNK_LEN = 256  # split input into chunks to save VRAM (shorter -> slower, but saves VRAM)

########################################################################################################

print(f"Loading model - {args.MODEL_NAME}")
model = RWKV(model=args.MODEL_NAME, strategy=args.strategy)
pipeline = PIPELINE(model, "rwkv_vocab_v20230424")


# def run_rnn(ctx, model_state=None):
#     ctx = ctx.replace("\r\n", "\n")
#     tokens = pipeline.encode(ctx)
#     tokens = [int(x) for x in tokens]
#     while len(tokens) > 0:
#         out, model_state = model.forward(tokens[:CHUNK_LEN], model_state)
#         tokens = tokens[CHUNK_LEN:]
#     return out, model_state


init_ctx = "User: hi" + "\n\n"
init_ctx += "Assistant: Hi. I am your assistant and I will provide expert full response in full details. Please feel free to ask any question and I will always answer it." + "\n\n"
init_ctx += "User:"
out0, INIT_STATE = model.forward(pipeline.encode(init_ctx), None)

# INIT_STATE = None

def response(msg, max_tokens=2000, temp=1.0, topp=0.3):
    msg = msg.strip()
    msg = re.sub(r"\n+", "\n", msg)

    if len(msg) == 0:
      raise ValueError("Please Say Something")
    model_state = None
    out, model_state = model.forward(pipeline.encode(" " + msg + "\n\nAssistant:"), copy.deepcopy(INIT_STATE))
    # print("\nAssistant:", end="")
    occurrence = {}
    out_tokens = []
    for i in range(max_tokens):
        for n in occurrence:
            out[n] -= GEN_alpha_presence + occurrence[n] * GEN_alpha_frequency # repetition penalty
        out[0] -= 1e10  # disable END_OF_TEXT
        token = pipeline.sample_logits(out, temperature=temp, top_p=topp)
        tmp = pipeline.decode(out_tokens)
        if "\n\n" in tmp:
          tmp=tmp.strip().strip("\n")
          print(tmp)
          print()
          return tmp
        out, model_state = model.forward([token], model_state)
        out_tokens += [token]

        for xxx in occurrence:
            occurrence[xxx] *= GEN_penalty_decay
        occurrence[token] = 1 + (occurrence[token] if token in occurrence else 0)
    tmp = pipeline.decode(out_tokens)
    tmp=tmp.strip().strip("\n")
    print(tmp)
    print()
    return tmp




questions = []

with open("/content/drive/MyDrive/data_release.jsonl", "r", encoding="UTF8") as fp:
    for line in fp.readlines():
        data = json.loads(line)
        questions.append((data["question"], data["category"]) )
j = 0

with open('/content/drive/MyDrive/example_rwkv6_1b6_chn_single.csv', 'w', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    for (q, r) in questions:
        j += 1
        print(j)
        q = q.strip()
        a = response(q, topp=(0.1 if r in ["逻辑推理","数学计算","专业能力"] else 0.7))
        writer.writerow([a])

